<center><h2>ALTEGRAD Project</h2>

<hr>
<span style="font-variant: small-caps;">Xavier Jiménez, Jean Quentin, Sacha Revol</span><br>
<hr>
</center>

# Classification from train/val data

## 1. Imports

In [ ]:
# XGboost
from xgboost import XGBClassifier

# Graphs
import networkx as nx

# Data
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

import pickle
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve
import sklearn
from sklearn.linear_model import LogisticRegression

import torch
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import torch.nn.functional as F


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MVA/ALTEGRAD/Final Project/')

In [ ]:
!ls

abstract_embeddings.ipynb			specter.ipynb
data						submission.csv
nn_classifier.ipynb				X_test_edge_embeddings.npy
sentence_transformer_abstract_embeddings.ipynb	X_train_val_edge_embeddings.npy
specter2.ipynb					X_val_edge_embeddings.npy


## 2. Loading data

### 2.1 Loading train/val

In [ ]:
#with open("data/training_data/X_val.npy", "rb") as fp:   #Pickling
with open("X_val_edge_embeddings.npy", "rb") as fp:   #Pickling
    X_val = np.load(fp)
    X_val = X_val.astype('float32')

with open("data/training_data/y_val.npy", "rb") as fp:   #Pickling
    y_val = np.load(fp)
    #y_val = y_val.astype('float32')

#with open("data/training_data/X_train_val.npy", "rb") as fp:   #Pickling
with open("X_train_val_edge_embeddings.npy", "rb") as fp:   #Pickling
    X_train_val = np.load(fp)
    X_train_val = X_train_val.astype('float32')

with open("data/training_data/y_train_val.npy", "rb") as fp:   #Pickling
    y_train_val = np.load(fp)
    #y_train_val = y_train_val.astype('float32')


## 3.Preprocessing

In [ ]:
X_val.shape

(69885, 1280)

## 4. Model

In [ ]:
import torch
import numpy as np

class Dataset(torch.utils.data.Dataset):

    def __init__(self, X_train, Y_train):
        
        #pos samples first
        self.labels = Y_train
        self.texts = X_train
        

        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        #print('getitem')
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y
    


In [ ]:
full_x = np.concatenate((X_val, X_train_val), axis=0)
full_y = np.concatenate((y_val, y_train_val), axis=0)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    full_x, full_y, test_size=0.05, random_state=42)


In [ ]:

train_data = Dataset(X_train_val, y_train_val)
val_data = Dataset(X_val, y_val)



In [ ]:
X_val.shape

(69885, 1280)

In [ ]:
#importing the libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 


batch_size = 512
#DataLoader
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)


#defining the network
from torch import nn
from torch.nn import functional as F
class Net(nn.Module):
    def __init__(self,input_shape):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_shape,640)
        #self.fc2 = nn.Linear(1280, 640)
        self.fc3 = nn.Linear(640, 320)
        self.fc4 = nn.Linear(320,160)
        self.fc5 = nn.Linear(160,80)
        self.fc6 = nn.Linear(80,20)
        self.fc7 = nn.Linear(20,1)
        #self.dropout = nn.Dropout(p=0.5)
        
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        #x = self.dropout(x)
        #x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        #x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        #x = self.dropout(x)
        x = torch.sigmoid(self.fc7(x))
        return x


#hyper parameters
epochs = 700
# Model , Optimizer, Loss
model = Net(input_shape=1280).double().to("cuda")

learning_rate = 0.0001
learning_rates = [0.01, 0.005, 0.0001, 0.00005, 0.00001]

from torch.optim.lr_scheduler import LambdaLR

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#scheduler = LambdaLR(optimizer, lr_lambda=[0.02, 0.01])
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=20,
 #                                             mode="triangular2", cycle_momentum=False)

#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

loss_fn = nn.BCELoss()


#forward loop
losses = []
accur = []
losses_val = []
accur_val = []

for i in range(epochs):

    #print("stepping!")
    #scheduler.step()

    losses = []
    accur = []
    losses_val = []
    accur_val = []

    model.train()
    for j,(x_train,y_train) in tqdm(enumerate(trainloader)):
        
        x_train = x_train.to("cuda")
        y_train = y_train.to("cuda")
        #calculate output
        output = model(x_train.double())

        #calculate loss
        loss = loss_fn(output , y_train.reshape(-1,1).double())

        #accuracy
        preds = (output.squeeze().detach().round())# == y_train)#.sum()
        acc = (torch.eq(preds, y_train).float().sum()/preds.shape[0]).item()
        
        losses.append(loss.item())
        accur.append(acc)

        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        
        
    with torch.no_grad():
        model.eval()

        for j,(x_test, y_test) in enumerate(test_loader):
        
                x_test = x_test.to("cuda")
                y_test = y_test.to("cuda")

                output = model(x_test.double())

                #calculate loss
                loss_val = loss_fn(output , y_test.reshape(-1,1).double())

                #accuracy
                preds = (output.squeeze().detach().round())# == y_train)#.sum()
                acc_val = (torch.eq(preds, y_test).float().sum()/preds.shape[0]).item()

                losses_val.append(loss_val.item())
                accur_val.append(acc_val)

    print("epoch {}\t train_loss : {}\t train_accuracy : {} \t val_loss : {}\t val_accuracy : {}"
          .format(i,
                  round(sum(losses)/len(losses),4),
                  round(sum(accur)/len(accur),4),
                  round(sum(losses_val)/len(losses_val),4),
                  round(sum(accur_val)/len(accur_val),4)))





## 5. Submission

In [ ]:
with open("X_test_edge_embeddings.npy", "rb") as fp:   #Pickling
    X_test_final = np.load(fp)
    X_test_final = X_test_final.astype('float32')

test_data_final = Dataset(X_test_final, y_train_val)


In [ ]:
X_test_final = torch.Tensor(X_test_final)


In [ ]:
model.eval()
with torch.no_grad():
  output = model(X_test_final.to("cuda").double())

In [ ]:
result_final = output.detach().to("cpu").numpy()

In [ ]:
result_final = np.squeeze(result_final, axis=1)



In [ ]:
result_final

array([0.99595408, 0.28561006, 0.97046264, ..., 0.05600482, 0.99826963,
       0.61759439])

In [ ]:
import csv 

predictions = zip(range(len(result_final)), result_final)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)
        